In [20]:
import os

import pymongo
from bson.json_util import dumps, loads
from bson.objectid import ObjectId

from splinter import Browser
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait # available since 2.4.0
from selenium.webdriver.support import expected_conditions as EC # available since 2.26.0
from selenium.common.exceptions import TimeoutException


import pandas as pd

import re
from pprint import pprint
from datetime import datetime
import time

import sys
sys.path.append('C:\\Users\\soria\\Documents\\WashUDataDocuments\\HwActivites\\YouTube_Project\\')
from config import YOUTUBE_DATA_API_KEY
import pandas as pd
import json
import os
from pprint import pprint

from apiclient.discovery import build
from apiclient.errors import HttpError
from oauth2client.tools import argparser



In [2]:
updated_asmr_df = pd.read_csv('asmr_merge_updated.csv')
updated_asmr_df['channel'] = [re.sub(r'\s+', "", channel) for channel in updated_asmr_df['channel']]
updated_asmr_df = updated_asmr_df.fillna(0)
updated_asmr_df.columns

Index(['Unnamed: 0', 'channel', 'url', 'rank', 'birthyear', 'age', 'uploads',
       'subs', 'total_views', 'channel_creation', 'gender', 'country',
       'category', 'twitter', 'instagram', 'twitch', 'facebook', 'type',
       'sb_url'],
      dtype='object')

In [3]:
channel_names = updated_asmr_df['channel'].values.tolist()
sb_urls = updated_asmr_df['sb_url'].values.tolist()
birthyears = updated_asmr_df['birthyear'].values.tolist()
genders = updated_asmr_df['gender'].values.tolist()
twitters = updated_asmr_df['twitter'].values.tolist()
instagrams = updated_asmr_df['instagram'].values.tolist()
twitches = updated_asmr_df['twitch'].values.tolist()
facebooks = updated_asmr_df['facebook'].values.tolist()
urls = updated_asmr_df['url'].values.tolist()

channel_names = [re.sub(r'\.+', '', name) for name in channel_names]
channel_names = [re.sub(r"'+", '', name) for name in channel_names]


In [4]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
mongoDb = client.asmr_youtube



In [7]:
executable_path = {'executable_path': '/Users/soria/Anaconda3/Scripts/chromedriver'}
# browser = Browser('chrome', **executable_path)
driver = webdriver.Chrome()


In [40]:
if mongoDb.youtube_scrape_response.find_one():
    for index, url in enumerate(urls[0:1]):
        channel = channel_names[index]
        collection = mongoDb.youtube_scrape_response
        html = loads(dumps(collection.find({channel : {'$exists':True}})))[0][channel]
        
        video_soup = bs(html, 'html.parser')
        video_soup.find_all('a', id='video-title')

        anchors = video_soup.find_all('a', id='video-title')
        videos = []
        for anchor in anchors:
            _id = anchor['href'][9:]
            video_dict = {'video_id': _id}
            videos.append(video_dict)
        collection = mongoDb.social_blade_asmr_data
        target = channel + '.videos'
        collection.update_one({}, {'$set': {target: videos} }, upsert=True)

else:
    youtube_asmr_response_dict = {}
    for index, url in enumerate(urls[0:1]):
        TIMEOUT_IN_SECONDS = 10

        wait = WebDriverWait(driver, TIMEOUT_IN_SECONDS)
        url = url + '/videos'
        driver.get(url)

        wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#items")))

        def get_count():
            return len(driver.find_elements_by_css_selector("#items #video-title"))

        while True:
            initial_count = get_count()
            driver.execute_script(
                "window.scrollTo(0, document.querySelector('#items').scrollHeight);")
            try:
                wait.until(
                    lambda _: get_count() > initial_count)
            except TimeoutException:
                # No additional content appeared. Abort our loop.
                break

        elements = driver.find_elements_by_css_selector("#items #video-title")

        html = driver.page_source

        channel = channel_names[index]
        youtube_asmr_response_dict.setdefault(channel, {})
        youtube_asmr_response_dict[channel] = html
        collection = mongoDb.youtube_scrape_response
        collection.update_one({}, {'$set': youtube_asmr_response_dict}, upsert=True)

        video_soup = bs(html, 'html.parser')
        video_soup.find_all('a', id='video-title')

        anchors = video_soup.find_all('a', id='video-title')
        videos = []
        for anchor in anchors:
            _id = anchor['href'][9:]
            video_dict = {'video_id': _id}
            videos.append(video_dict)
        collection = mongoDb.social_blade_asmr_data
        target = channel + '.videos'
        collection.update_one({}, {'$set': {target: videos} }, upsert=True)

In [170]:
def topicCipher(array):
    df = pd.DataFrame({'topic_id': array})
    merge = pd.merge(df, topic_df, on='topic_id', how='left')
    return merge['topic_name'].values.tolist()
def catCipher(array):
    df = pd.DataFrame({'category_id': array})
    merge = pd.merge(df, topic_df, on='category_id', how='left')
    return merge['category_id'].values.tolist()
   
            

In [45]:
DEVELOPER_KEY = YOUTUBE_DATA_API_KEY
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"


In [46]:
youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
    developerKey=DEVELOPER_KEY)

In [161]:
if mongoDb.social_blade_asmr_data.find_one():
    for index, url in enumerate(urls[0:1]):
        
        channel = channel_names[index]
        collection = mongoDb.social_blade_asmr_data
        
        old_video_dict_list = loads(dumps(collection.find({channel : {'$exists':True}})))[0][channel]['videos']
        new_video_dict_list = []
        for video in old_video_dict_list[0:1]:
        
            video_search_response = youtube.videos().list(
                id=video['video_id'],
                pageToken=None,
                part="snippet,contentDetails,statistics,topicDetails,id",
                maxResults=50


            ).execute()
            published_at = video_search_response['items'][0]['snippet']['publishedAt']
            video_id = video_search_response['items'][0]['id']
            title = video_search_response['items'][0]['snippet']['title']
            category_id = video_search_response['items'][0]['snippet']['categoryId']
            comment_count = video_search_response['items'][0]['statistics']['commentCount']
            dislike_count = video_search_response['items'][0]['statistics']['dislikeCount']
            like_count = video_search_response['items'][0]['statistics']['likeCount']
            topic_ids = video_search_response['items'][0]['topicDetails']['topicIds']
            relevant_topic_ids = video_search_response['items'][0]['topicDetails']['relevantTopicIds']
            video_dict = {
                'video_id': video_id, 'title': title, 'published_at': published_at, 'comment_count': comment_count,
                'like_count': like_count, 'dislike_count': dislike_count, 'category_id': category_id,
                'topic_ids': topicCipher(topic_ids), 'relevant_topic_ids': topicCipher(relevant_topic_ids)
            }
            new_video_dict_list.append(video_dict)
        collection = mongoDb.social_blade_asmr_data
        target = channel + '.videos'
        collection.update_one({}, {'$set': {target: new_video_dict_list} }, upsert=True)

   topic_id
0  /m/064t9
1  /m/04rlf
   topic_id topic_heading  topic_name
0  /m/064t9  Music topics   Pop music
1  /m/04rlf  Music topics       Music
   topic_id
0  /m/04rlf
1  /m/064t9
   topic_id topic_heading  topic_name
0  /m/04rlf  Music topics       Music
1  /m/064t9  Music topics   Pop music


In [169]:
resp = video_search_response
resp

{'kind': 'youtube#videoListResponse',
 'etag': '"XpPGQXPnxQJhLgs6enD_n8JR4Qk/zsL4DE0YrepkbabIf1gSrkBSQqY"',
 'pageInfo': {'totalResults': 1, 'resultsPerPage': 1},
 'items': [{'kind': 'youtube#video',
   'etag': '"XpPGQXPnxQJhLgs6enD_n8JR4Qk/UMGcqeC168PZNa5BUjzQ9pmxCXY"',
   'id': 'RDfiVrHTJNM',
   'snippet': {'publishedAt': '2019-02-05T00:24:10.000Z',
    'channelId': 'UCN1hnUccO4FD5WfM7ithXaw',
    'title': 'Maroon 5 - Pepsi Super Bowl LIII Halftime Show ft. Travis Scott, Big Boi',
    'description': '"RED PILL BLUES” is out now.\nhttp://smarturl.it/RedPillBlues\n \nFor more, visit:\nhttps://www.facebook.com/maroon5\nhttps://twitter.com/maroon5\nhttps://www.instagram.com/maroon5\n \nSign up for updates: http://smarturl.it/Maroon5.News\n\nhttp://vevo.ly/sjkPlH',
    'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/RDfiVrHTJNM/default.jpg',
      'width': 120,
      'height': 90},
     'medium': {'url': 'https://i.ytimg.com/vi/RDfiVrHTJNM/mqdefault.jpg',
      'width': 320,
   

In [81]:

collection = mongoDb.topicId_gist_html
if mongoDb.topicId_gist_html.find_one():
    html = mongoDb.topicId_gist_html.find_one()['gist_html']
else:
    url = "https://gist.github.com/stpe/2951130dfc8f1d0d1a2ad736bef3b703"
    driver = webdriver.Chrome()
    driver.get(url)
    wait = WebDriverWait(driver, 10)
    element = wait.until(EC.element_to_be_clickable((By.ID,'user-content-other-topics')))
    html = driver.page_source
topic_id_html_dict = {'topicId_gist_url': driver.current_url, 'gist_html':html}
collection.update_one({}, {'$set': topic_id_html_dict}, upsert=True)
topic_soup = bs(html, 'html.parser')
heading_list = []
topic_headings = topic_soup.find_all('h2')
for heading in topic_headings:
    heading_list.append(heading.text)
topics_dict = {}
topic_dict_list = []
topics = topic_soup.find_all('li')
for index, topic in enumerate(topics[4:20]):
    text = topic.text
    topic_tuple = re.split(r'\s\s\s', text)
    collection = mongoDb.youtube_topicIds
    topic_name = topic_tuple[1]
    topic_id = topic_tuple[0]
    topic_heading = heading_list[0]
    topics_dict.setdefault(topic_heading, [])
    topics_dict[topic_heading].append({'topic_name': topic_name, 'topic_id': topic_id, 'topic_heading': topic_heading})
    topic_dict_list.append({'topic_name': topic_name, 'topic_id': topic_id, 'topic_heading': topic_heading})
    collection.update_one({}, {'$set': topics_dict}, upsert=True)
    
for index, topic in enumerate(topics[20:31]):
    text = topic.text
    topic_tuple = re.split(r'\s\s\s', text)
    collection = mongoDb.youtube_topicIds
    topic_name = topic_tuple[1]
    topic_id = topic_tuple[0]
    topic_heading = heading_list[1]
    topics_dict.setdefault(topic_heading, [])
    topics_dict[topic_heading].append({'topic_name': topic_name, 'topic_id': topic_id, 'topic_heading': topic_heading})
    topic_dict_list.append({'topic_name': topic_name, 'topic_id': topic_id, 'topic_heading': topic_heading})
    collection.update_one({}, {'$set': topics_dict}, upsert=True)
    
for index, topic in enumerate(topics[31:45]):
    text = topic.text
    topic_tuple = re.split(r'\s\s\s', text)
    collection = mongoDb.youtube_topicIds
    topic_name = topic_tuple[1]
    topic_id = topic_tuple[0]
    topic_heading = heading_list[2]
    topics_dict.setdefault(topic_heading, [])
    topics_dict[topic_heading].append({'topic_name': topic_name, 'topic_id': topic_id, 'topic_heading': topic_heading}) 
    topic_dict_list.append({'topic_name': topic_name, 'topic_id': topic_id, 'topic_heading': topic_heading})
    collection.update_one({}, {'$set': topics_dict}, upsert=True)
    
for index, topic in enumerate(topics[45:50]):
    text = topic.text
    topic_tuple = re.split(r'\s\s\s', text)
    collection = mongoDb.youtube_topicIds
    topic_name = topic_tuple[1]
    topic_id = topic_tuple[0]
    topic_heading = heading_list[3]
    topics_dict.setdefault(topic_heading, [])
    topics_dict[topic_heading].append({'topic_name': topic_name, 'topic_id': topic_id, 'topic_heading': topic_heading}) 
    topic_dict_list.append({'topic_name': topic_name, 'topic_id': topic_id, 'topic_heading': topic_heading})
    collection.update_one({}, {'$set': topics_dict}, upsert=True)
    
for index, topic in enumerate(topics[50:61]):
    text = topic.text
    topic_tuple = re.split(r'\s\s\s', text)
    collection = mongoDb.youtube_topicIds
    topic_name = topic_tuple[1]
    topic_id = topic_tuple[0]
    topic_heading = heading_list[4]
    topics_dict.setdefault(topic_heading, [])
    topics_dict[topic_heading].append({'topic_name': topic_name, 'topic_id': topic_id, 'topic_heading': topic_heading}) 
    topic_dict_list.append({'topic_name': topic_name, 'topic_id': topic_id, 'topic_heading': topic_heading})
    collection.update_one({}, {'$set': topics_dict}, upsert=True)
    
for index, topic in enumerate(topics[61:63]):
    text = topic.text
    topic_tuple = re.split(r'\s\s\s', text)
    collection = mongoDb.youtube_topicIds
    topic_name = topic_tuple[1]
    topic_id = topic_tuple[0]
    topic_heading = heading_list[5]
    topics_dict.setdefault(topic_heading, [])
    topics_dict[topic_heading].append({'topic_name': topic_name, 'topic_id': topic_id, 'topic_heading': topic_heading})
    topic_dict_list.append({'topic_name': topic_name, 'topic_id': topic_id, 'topic_heading': topic_heading})
    collection.update_one({}, {'$set': topics_dict}, upsert=True)
topics_dict['all_topics'] = topic_dict_list
collection.update_one({}, {'$set': topics_dict}, upsert=True)

In [166]:
collection = mongoDb.regionCode_ytApi_response
if mongoDb.regionCode_ytApi_resonse.find_one():
    region_code_search_response = mongoDb.regionCode_ytApi_response.find_one()['region_code_ytApi_response']
else:
    region_code_search_response = youtube.i18nRegions().list(
    part="snippet,id"
        
    ).execute()
region_code_response_dict = {'region_code_ytApi_response': region_code_search_response}
collection.update_one({}, {'$set': region_code_response_dict}, upsert=True)
region_code_dict_list = []
region_codes_dict = {"region_codes": []}
for item in region_code_search_response['items']:
    region = item['snippet']['name']
    region_code = item['snippet']['gl']
    region_codes_dict['region_codes'].append({'region': region, 'region_code': region_code})
    collection = mongoDb.youtube_regionCodes
    collection.update_one({}, {'$set': region_codes_dict}, upsert=True)
    region_code_dict_list.append({'region': region, 'region_code': region_code})
    


In [168]:
collection = mongoDb.videoCategory_ytApi_response
if mongoDb.videoCategory_ytApi_response.find_one():
    all_region_category_list = []
    video_category_response_dict = {}
    region_categories_dict = {}
    for region in region_code_dict_list:
        region_code = region['region_code']
        collection = mongoDb.videoCategory_ytApi_response
        video_category_search_response = loads(dumps(collection.find({region_code : {'$exists':True}})))
        for response in video_category_search_response:
            region_categories_dict.setdefault(region_code, [])
            region_category_list = []
            for item in response[region_code]['items']:
                category_title = item['snippet']['title']
                category_id = item['id']
                category_is_assignable = item['snippet']['assignable']
                category_id_dict = {'category_title': category_title, 'category_id': category_id,\
                                    'category_is_assignable': category_is_assignable, 'region_code': region_code}
                region_category_list.append(category_id_dict)
                region_categories_dict[region_code].append(category_id_dict)
            collection = mongoDb.youtube_videoCategories
            collection.update_one({}, {'$set': region_categories_dict}, upsert=True)
            region['categories'] = region_category_list
            all_region_category_list.append(region_category_list)
else:
    all_region_category_list = []
    video_category_response_dict = {}
    region_categories_dict = {}
    for region in region_code_dict_list:
        region_code = region['region_code']
        video_category_search_response = youtube.videoCategories().list(
            regionCode=region_code,
            part="snippet,id"
        ).execute()
        video_category_response_dict.setdefault(region_code, {})
        video_category_response_dict[region_code] = video_category_search_response
        collection = mongoDb.videoCategory_ytApi_response
        collection.update_one({}, {'$set': video_category_response_dict}, upsert=True)
        region_categories_dict.setdefault(region_code, [])
        region_category_list = []
        for item in video_category_search_response['items']:
            category_title = item['snippet']['title']
            category_id = item['id']
            category_is_assignable = item['snippet']['assignable']
            category_id_dict = {'category_title': category_title, 'category_id': category_id,\
                                'category_is_assignable': category_is_assignable, 'region_code': region_code}
            region_category_list.append(category_id_dict)
            region_categories_dict[region_code].append(category_id_dict)
        collection = mongoDb.youtube_videoCategories
        collection.update_one({}, {'$set': region_categories_dict}, upsert=True)
        region['categories'] = region_category_list
        all_region_category_list.append(region_category_list)
